# Light Curve Analysis

This notebook demonstrates:
1. Source detection and photometry
2. Differential photometry for exoplanet transits
3. Light curve visualization and analysis

## Prerequisites
- Reduced FITS files in `../data/reduced/`
- Completed data reduction (notebook 01)

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

from src.data_reduction import load_fits
from src.photometry import (
    find_sources,
    measure_aperture_photometry,
    extract_light_curve
)
from src.utils import list_fits_files, plot_image, plot_light_curve

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Load Reduced Images

In [ ]:
# Directory paths
reduced_dir = Path('../data/reduced')

# List reduced FITS files
reduced_files = sorted(list_fits_files(reduced_dir))
print(f"Found {len(reduced_files)} reduced frames")

if len(reduced_files) == 0:
    print("\n⚠ No reduced frames found!")
    print("Please run notebook 01_data_reduction.ipynb first.")
else:
    # Load first frame for source detection
    data, header = load_fits(reduced_files[0])
    print(f"\nImage shape: {data.shape}")
    print(f"Data range: [{np.min(data):.1f}, {np.max(data):.1f}]")

## 2. Source Detection

Detect stars in the field using DAOStarFinder.

In [ ]:
if len(reduced_files) > 0:
    # Detect sources
    sources = find_sources(data, fwhm=5.0, threshold=5.0)
    
    print(f"Detected {len(sources)} sources")
    print("\nBrightest sources:")
    sources_sorted = sources.copy()
    sources_sorted.sort('flux')
    sources_sorted.reverse()
    print(sources_sorted[:10]['id', 'xcentroid', 'ycentroid', 'flux'])
    
    # Display image with detected sources
    fig, ax = plt.subplots(figsize=(12, 10))
    plot_image(data, title='Detected Sources', ax=ax)
    ax.scatter(sources['xcentroid'], sources['ycentroid'], 
              s=100, facecolors='none', edgecolors='r', linewidths=1.5)
    
    # Label brightest sources
    for i, source in enumerate(sources_sorted[:5]):
        ax.text(source['xcentroid']+10, source['ycentroid'], 
               f"{i+1}", color='red', fontsize=12, weight='bold')
    
    plt.tight_layout()
    plt.show()

## 3. Select Target and Comparison Stars

**Instructions:**
1. Identify your target star from the plot above
2. Select stable comparison stars (bright, unsaturated)
3. Enter their positions below

In [ ]:
# Example: manually specify star positions
# Replace these with actual coordinates from your images

if len(reduced_files) > 0 and len(sources) > 0:
    # Use brightest source as example target
    target_position = (sources_sorted[0]['xcentroid'], 
                      sources_sorted[0]['ycentroid'])
    
    # Use next 3 bright sources as comparisons
    comparison_positions = [
        (sources_sorted[1]['xcentroid'], sources_sorted[1]['ycentroid']),
        (sources_sorted[2]['xcentroid'], sources_sorted[2]['ycentroid']),
        (sources_sorted[3]['xcentroid'], sources_sorted[3]['ycentroid'])
    ]
    
    print(f"Target position: ({target_position[0]:.1f}, {target_position[1]:.1f})")
    print(f"\nComparison stars:")
    for i, pos in enumerate(comparison_positions):
        print(f"  {i+1}: ({pos[0]:.1f}, {pos[1]:.1f})")

## 4. Aperture Photometry

Measure flux in apertures around stars.

In [ ]:
if len(reduced_files) > 0 and len(sources) > 0:
    # Perform photometry on first frame
    aperture_radius = 10  # pixels, adjust based on FWHM
    
    phot_table = measure_aperture_photometry(
        data, 
        [target_position] + comparison_positions,
        aperture_radius=aperture_radius
    )
    
    print("Photometry Results:")
    print(phot_table['xcenter', 'ycenter', 'aper_sum_bkgsub', 'mag_inst'])
    
    # Visualize apertures
    from photutils.aperture import CircularAperture
    
    fig, ax = plt.subplots(figsize=(12, 10))
    plot_image(data, title='Photometry Apertures', ax=ax)
    
    # Target in red
    target_ap = CircularAperture([target_position], r=aperture_radius)
    target_ap.plot(ax=ax, color='red', lw=2, label='Target')
    
    # Comparisons in cyan
    comp_ap = CircularAperture(comparison_positions, r=aperture_radius)
    comp_ap.plot(ax=ax, color='cyan', lw=2, label='Comparison')
    
    ax.legend(loc='upper right')
    plt.tight_layout()
    plt.show()

## 5. Extract Light Curve

Perform differential photometry on all frames.

In [ ]:
if len(reduced_files) > 0 and len(sources) > 0:
    print("Extracting light curve...")
    
    lightcurve = extract_light_curve(
        reduced_files,
        target_position=target_position,
        comparison_positions=comparison_positions,
        aperture_radius=aperture_radius
    )
    
    print(f"\nLight curve with {len(lightcurve)} points")
    print(lightcurve.head())
    
    # Save to CSV
    output_file = '../data/lightcurve.csv'
    lightcurve.to_csv(output_file, index=False)
    print(f"\nSaved to {output_file}")

## 6. Plot Light Curve

In [ ]:
if len(reduced_files) > 0 and len(sources) > 0:
    # Normalize time to hours from first observation
    time_hours = (lightcurve['time'] - lightcurve['time'].iloc[0]) * 24
    
    # Plot relative flux
    fig, ax = plot_light_curve(
        time_hours,
        lightcurve['relative_flux'],
        title='Differential Light Curve',
        xlabel='Time (hours)',
        ylabel='Relative Flux'
    )
    
    # Add reference line at 1.0
    ax.axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='Baseline')
    ax.legend()
    
    plt.savefig('../data/lightcurve.png', dpi=150, bbox_inches='tight')
    print("Saved plot to ../data/lightcurve.png")
    plt.show()
    
    # Statistics
    print(f"\nLight curve statistics:")
    print(f"  Mean flux: {lightcurve['relative_flux'].mean():.6f}")
    print(f"  Std dev: {lightcurve['relative_flux'].std():.6f}")
    print(f"  RMS: {np.sqrt(np.mean(lightcurve['relative_flux']**2)):.6f}")
    print(f"  Min flux: {lightcurve['relative_flux'].min():.6f}")
    print(f"  Max flux: {lightcurve['relative_flux'].max():.6f}")

## 7. Transit Detection (Optional)

Look for periodic dips in the light curve that might indicate a transit.

In [ ]:
if len(reduced_files) > 0 and len(sources) > 0:
    # Simple transit detection: look for significant dips
    median_flux = lightcurve['relative_flux'].median()
    std_flux = lightcurve['relative_flux'].std()
    
    # Points more than 3-sigma below median
    threshold = median_flux - 3 * std_flux
    transit_candidates = lightcurve[lightcurve['relative_flux'] < threshold]
    
    if len(transit_candidates) > 0:
        print(f"Found {len(transit_candidates)} potential transit points:")
        print(transit_candidates[['time', 'relative_flux']])
    else:
        print("No significant transit events detected.")
        print(f"(Threshold: {threshold:.6f}, Min observed: {lightcurve['relative_flux'].min():.6f})")

## Summary

✓ Sources detected and photometry performed
✓ Light curve extracted and saved
✓ Differential photometry completed

**Next Steps:**
- Analyze light curve for transit features
- Fit transit model if transit detected
- Proceed to `03_standard_stars.ipynb` for calibration
- Export results for manuscript